# Essential imports 

In [1]:
import requests
import pandas as pd
import json
import numpy as np

# Prepare the *headers* for the API request call with credentials

In [2]:
import time
headers = {
    'accept': 'application/json',
    'X-IBM-Client-Id': 'fa952a57-3114-47bd-81ef-b0a46b572999',   # ibm-id lavmlk20201@gmail.com
    'X-IBM-Client-Secret' : 'A7eX4qT0jP1pU2oS4bE0dH5cL8xF8oI7pN5dT7kY6vB7oY0vI1'   
}

# Preliminary *health check* to make sure your credentials are OK and you can access the service!

In [3]:
base_url = "https://api.ibm.com/"
health_check = "timeseriesclass/run/timeseriesclassification/health_check"
full_health_check_url = base_url + health_check
print(full_health_check_url)
response = requests.get(full_health_check_url, headers=headers)
print(response.text)

https://api.ibm.com/timeseriesclass/run/timeseriesclassification/health_check
"Success"



# Prepare *parameters* and *files* inputs for the HTTP request for accessing the NN Rocket service

In [21]:
params = (
    ('time column', 'Time'),
    ('time string format', '%Y-%m-%d %H:%M:%S'),
    ('target columns', '["Value1","Value2","Value3","Value4","Value5","Value6"]'),
    ('categorical columns', '["Category1","Category2","Category3"]'),
    ('label column', 'Label'),
    ('snapshot column', 'Snapshot'),
    ('train test split', '0.5'),
    ('result type', 'accuracy'),
)
files = {'data file': open("user_df_tracking.csv", "rb")}

# Your data file in CSV format & various validation checks, prior to making the HTTP request
> ## The *validator* function below makes several basic sanity checks to help you avoid surprises

In [22]:
data_file = "user_df_tracking.csv"   # "sample.csv"
params_dict = dict(params)
time_column = params_dict['time column']
time_format = params_dict['time string format']
target_columns = json.loads(params_dict['target columns'])
label_column =  params_dict['label column']
snapshot_column = params_dict['snapshot column']

def validator(data_file, time_column, time_format, target_columns, label_column, snapshot_column):

    try:
        userdf = pd.read_csv(data_file)
    except FileNotFoundError:
        raise FileNotFoundError("Submitted csv file not found.")
    except EmptyDataError:
        raise EmptyDataError("There is no data in the submitted csv")
    except ParserError:
        raise ParserError("There is an issue parsing the submitted csv")

    # check column headings
    column_names = userdf.columns
    if time_column not in column_names:
        raise ValueError(time_column+'  not found in submitted csv file')
    if label_column not in column_names:
        raise ValueError(label_column+'  not found in submitted csv file')
    if snapshot_column not in column_names:
        raise ValueError(snapshot_column+'  not found in submitted csv file')
    # check target columns exist and is numeric 
    for t in target_columns:
        if t not in column_names:
            raise ValueError(t+'  not found in submitted csv file')
        else:
            if not userdf[t].apply(np.isreal).all(axis=None):
                raise ValueError('Non numeric values found in column ',t)

    # check time formatting
    try:
        userdf[time_column] = pd.to_datetime(userdf[time_column],format=time_format)
    except ValueError as e:
        raise ValueError('Specified time column not recognized'+e)

    # check distinct timestamps
    uniquetscount = userdf[[snapshot_column, time_column]].groupby([snapshot_column]).nunique()[time_column].tolist() 
    numtscount = userdf[[snapshot_column, time_column]].groupby([snapshot_column]).size().tolist()
    if uniquetscount != numtscount:
        raise ValueError('Duplicate timestamps detected in the csv')

    # check label column for single cases
    groupbylabel = userdf[[snapshot_column, label_column]].groupby(snapshot_column).mean()
    labelid = groupbylabel[label_column]
    if labelid.value_counts(ascending=True).tolist()[0] < 5:
        raise ValueError('One or more labels have less than five snapshot instances and thus insufficient training sample')

    return userdf

user_df = validator(data_file, time_column, time_format, target_columns, label_column, snapshot_column)

# Inspect data prior to sending it across

In [6]:
user_df.head()

,Snapshot,Label,Time,Value1,Value2,Value3,Value4,Value5,Value6,Category1,Category2,Category3
0,0,0,2025-09-21 19:00:00,0.079106,0.394032,0.551444,0.351565,0.023970,0.633883,A,A,E
1,0,0,2025-09-21 20:00:00,0.079106,0.394032,0.551444,0.351565,0.023970,0.633883,A,B,E
2,0,0,2025-09-21 21:00:00,-0.903497,-3.666397,-0.282844,-0.095881,-0.319605,0.972131,A,C,E
3,0,0,2025-09-21 22:00:00,1.116125,-0.656101,0.333118,1.624657,-0.569962,1.209171,A,C,E
4,0,0,2025-09-21 23:00:00,1.638200,1.405135,0.393875,1.187864,-0.271664,1.739182,A,C,E


In [7]:
user_df.Snapshot.value_counts().describe()

# Snapshot is 80 different values 100 times each --> equivalent to the series_id column in eye-tracking data after exploting dataframe

count     80.0
mean     100.0
std        0.0
min      100.0
25%      100.0
50%      100.0
75%      100.0
max      100.0
Name: Snapshot, dtype: float64

In [19]:
# GazeDefy

# adapt dataframe to fit NNRocket user_df

user_df_eye_tracking = pd.read_csv("exploted_gazedefy.csv")
user_df_eye_tracking.columns = ["Snapshot",	"Label",	"Time",	"Value1",	"Value2",	"Value3",	"Value4",	"Value5",	"Value6",	"Category1"]
party_dict = {"Anoth": 1,
           "Arif": 2,
           "Ashok": 3,
           "Gowthom": 4,
           "Josephin": 5,
           "Raghu": 6,
           "Sebastian": 7,
           }
user_df_eye_tracking.replace({"Label": party_dict}, inplace=True)
user_df_eye_tracking

,Snapshot,Label,Time,Value1,Value2,Value3,Value4,Value5,Value6,Category1
0,0,1,0,0.023858,2.253397,0.2780,0.1360,67,2.308106,2
1,0,1,1,0.023858,2.253397,0.2780,0.1360,67,2.308106,2
2,0,1,2,0.023858,2.253397,0.2780,0.1360,67,2.308106,2
3,0,1,3,0.023858,2.209508,0.2780,0.1360,67,2.308106,2
4,0,1,4,0.023858,2.134531,0.2780,0.1360,67,2.308106,2
...,...,...,...,...,...,...,...,...,...,...
99493,629,7,156,-0.324802,1.723117,0.6556,0.8524,67,1.769930,3
99494,629,7,157,-0.022118,2.093707,0.6622,0.8540,67,1.769930,3
99495,629,7,158,0.066907,2.299353,0.6728,0.8547,67,1.769930,3
99496,629,7,159,-0.075533,2.282215,0.6889,0.8538,67,1.769930,3


In [20]:
user_df_eye_tracking.to_csv("user_df_tracking.csv", index=None)

In [23]:
user_df.Value5.value_counts().describe()

count     101.000000
mean      985.128713
std       872.461697
min       161.000000
25%       161.000000
50%       644.000000
75%      1610.000000
max      4347.000000
Name: Value5, dtype: float64

In [16]:
user_df_eye_tracking.Snapshot.dtype

dtype('int64')

# Make the HTTP request using the *requests* module
> ## It returns a *task_id*

In [24]:
nnrocket_svc = "timeseriesclass/run/timeseriesclassification/nnrocket"

full_svc_url = base_url+nnrocket_svc
print(full_svc_url)
response = requests.post(full_svc_url, headers=headers, params=params, files=files)
task_id = response.text
task_id=task_id.replace('""','').rstrip()
print(task_id)


https://api.ibm.com/timeseriesclass/run/timeseriesclassification/nnrocket
"e51ae121334a4573b8ccd9bfcc763868"


# Make a status check call against the above returned *task_id* 

In [25]:
status_check_svc = "timeseriesclass/run/timeseriesclassification/status/"
full_status_check_url = base_url+status_check_svc + task_id.replace('"', '')
print(full_status_check_url)
response = requests.get(full_status_check_url, headers=headers)
print(response.text)

https://api.ibm.com/timeseriesclass/run/timeseriesclassification/status/e51ae121334a4573b8ccd9bfcc763868
"task is running.."



# Check Again! Using the same *task_id* 

In [26]:
response = requests.get(full_status_check_url, headers=headers)
print(response.text)

label,precision,recall,f1-score,support
1,0.8863636363636364,0.8666666666666667,0.8764044943820225,45.0
2,0.7547169811320755,0.9090909090909091,0.8247422680412373,44.0
3,0.7391304347826086,0.8292682926829268,0.7816091954022988,41.0
4,0.7555555555555555,0.7555555555555555,0.7555555555555555,45.0
5,0.9302325581395349,0.9090909090909091,0.9195402298850575,44.0
6,0.8,0.7111111111111111,0.7529411764705882,45.0
7,0.8947368421052632,0.7555555555555555,0.8192771084337349,45.0
accuracy,0.8187702265372169,0.8187702265372169,0.8187702265372169,0.8187702265372169
macro avg,0.8229622868683819,0.8194769999648048,0.818581432595785,309.0
weighted avg,0.8239211944502858,0.8187702265372169,0.8187133721298925,309.0



In [28]:
response = requests.get(full_status_check_url, headers=headers)
print(response.text)

label,precision,recall,f1-score,support
1,0.8863636363636364,0.8666666666666667,0.8764044943820225,45.0
2,0.7547169811320755,0.9090909090909091,0.8247422680412373,44.0
3,0.7391304347826086,0.8292682926829268,0.7816091954022988,41.0
4,0.7555555555555555,0.7555555555555555,0.7555555555555555,45.0
5,0.9302325581395349,0.9090909090909091,0.9195402298850575,44.0
6,0.8,0.7111111111111111,0.7529411764705882,45.0
7,0.8947368421052632,0.7555555555555555,0.8192771084337349,45.0
accuracy,0.8187702265372169,0.8187702265372169,0.8187702265372169,0.8187702265372169
macro avg,0.8229622868683819,0.8194769999648048,0.818581432595785,309.0
weighted avg,0.8239211944502858,0.8187702265372169,0.8187133721298925,309.0

